In [ ]:
!pip install google_trans_new

In [ ]:
import random
import google_trans_new
from google_trans_new import google_translator 

In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [ ]:
import pandas as pd
df = pd.read_excel('/content/tweets_-44724447.xlsx')
df.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [ ]:
df.shape

(1364, 2)

In [ ]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [ ]:
# Import Library
import random
# import torch, torchtext
from torchtext import data 
import torch
from torchtext.legacy import data
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [ ]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [ ]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [ ]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

# Data Augmentatiob Process - convert the train sample generator to train data frame before applying the augmentation process

In [ ]:
## Convert the train sample generator into train dataframe

tweets=[]
labels=[]
for x in train.examples:
  tweets.append(vars(x).get('tweets'))
  labels.append(vars(x).get('labels'))

df_train=pd.DataFrame({'Tweets':tweets,'Labels':labels})
# print(tweets)
print('\n')
# print(labels)



0    797
1    293
2     69
Name: Labels, dtype: int64


# DATA Augmentation process

In [ ]:
## Perform data augmentation on train data frame
# Random Delete
# Random Swap
# Back Translate

from tqdm import tqdm
import random
from sklearn.utils import shuffle
# Augmenting 20% of data

samples_0=int(df_train['Labels'].value_counts()[0]*.2)
samples_1=int(df_train['Labels'].value_counts()[1]*.2)
samples_2=int(df_train['Labels'].value_counts()[2]*.2)


    
##selecting the class samples
df_0=df_train[df_train.Labels==0].reset_index(drop=True)
df_1=df_train[df_train.Labels==1].reset_index(drop=True)
df_2=df_train[df_train.Labels==2].reset_index(drop=True)

## Random Delete ####

new_text=[]
## data augmentation loop - class 0
for i in tqdm(np.random.randint(0,len(df_0),samples_0)):
  text = df_0.iloc[i]['Tweets']
  augmented_text = random_deletion(text,0.1)
  new_text.append(augmented_text)
## dataframe
rd_new_0=pd.DataFrame({'Tweets':new_text,'Labels':0})

new_text=[]
## data augmentation loop - class 1
for i in tqdm(np.random.randint(0,len(df_1),samples_1)):
  text = df_1.iloc[i]['Tweets']
  augmented_text = random_deletion(text,0.1)
  new_text.append(augmented_text)
## dataframe
rd_new_1=pd.DataFrame({'Tweets':new_text,'Labels':1})

new_text=[]
## data augmentation loop - class 2
for i in tqdm(np.random.randint(0,len(df_2),samples_2)):
  text = df_2.iloc[i]['Tweets']
  augmented_text = random_deletion(text,0.1)
  new_text.append(augmented_text)
## dataframe
rd_new_2=pd.DataFrame({'Tweets':new_text,'Labels':2})

print('\n',rd_new_0.shape,rd_new_1.shape,rd_new_2.shape)


## Random Swap ####

new_text=[]
## data augmentation loop - class 0
for i in tqdm(np.random.randint(0,len(df_0),samples_0)):
  text = df_0.iloc[i]['Tweets']
  augmented_text = random_swap(text)
  new_text.append(augmented_text)
## dataframe
rs_new_0=pd.DataFrame({'Tweets':new_text,'Labels':0})

new_text=[]
## data augmentation loop - class 1
for i in tqdm(np.random.randint(0,len(df_1),samples_1)):
  text = df_1.iloc[i]['Tweets']
  augmented_text = random_swap(text)
  new_text.append(augmented_text)
## dataframe
rs_new_1=pd.DataFrame({'Tweets':new_text,'Labels':1})

new_text=[]
## data augmentation loop - class 2
for i in tqdm(np.random.randint(0,len(df_2),samples_2)):
  text = df_2.iloc[i]['Tweets']
  augmented_text = random_swap(text)
  new_text.append(augmented_text)
## dataframe
rs_new_2=pd.DataFrame({'Tweets':new_text,'Labels':2})

print('\n',rs_new_0.shape,rs_new_1.shape,rs_new_2.shape)


## Back Translate ####

translator=google_translator()
available_langs = list(google_trans_new.LANGUAGES.keys()) 

new_text=[]
## data augmentation loop - class 0
for i in tqdm(np.random.randint(0,len(df_0),samples_0)):
  trans_lang = random.choice(available_langs)
  text = df_0.iloc[i]['Tweets']
  translations=translator.translate(text, lang_tgt=trans_lang)
  augmented_text=translator.translate(translations, lang_src=trans_lang, lang_tgt='en')
  new_text.append(augmented_text)
## dataframe
bt_new_0=pd.DataFrame({'Tweets':new_text,'Labels':0})

new_text=[]
## data augmentation loop - class 1
for i in tqdm(np.random.randint(0,len(df_1),samples_1)):
  trans_lang = random.choice(available_langs)
  text = df_1.iloc[i]['Tweets']
  translations=translator.translate(text, lang_tgt=trans_lang)
  augmented_text=translator.translate(translations, lang_src=trans_lang, lang_tgt='en')
  new_text.append(augmented_text)
## dataframe
bt_new_1=pd.DataFrame({'Tweets':new_text,'Labels':1})

new_text=[]
## data augmentation loop - class 2
for i in tqdm(np.random.randint(0,len(df_2),samples_2)):
  trans_lang = random.choice(available_langs)
  text = df_2.iloc[i]['Tweets']
  translations=translator.translate(text, lang_tgt=trans_lang)
  augmented_text=translator.translate(translations, lang_src=trans_lang, lang_tgt='en')
  new_text.append(augmented_text)
## dataframe
bt_new_2=pd.DataFrame({'Tweets':new_text,'Labels':2})

print('\n',bt_new_0.shape,bt_new_1.shape,bt_new_2.shape)




  0%|          | 0/159 [00:00<?, ?it/s]


 (159, 2) (58, 2) (13, 2)

 (159, 2) (58, 2) (13, 2)


100%|██████████| 13/13 [00:10<00:00,  1.26it/s]


 (159, 2) (58, 2) (13, 2)


In [ ]:
df_train_new=shuffle(df_train.append([rd_new_0,rd_new_1,rd_new_2,rs_new_0,rs_new_1,rs_new_2,bt_new_0,bt_new_1,bt_new_2]).reset_index(drop=True)).reset_index(drop=True)
print('\n',df_train.head(),'\n',df_train_new.head(),'\n',df_train.shape,'\n', df_train_new.shape )



                                               Tweets  Labels
0  [@billmaher, Mr, Maher, I, just, seen, the, HB...       0
1  [Elections, #, of, @TBCDG, the, #, tcot, -, !,...       0
2  [Huffington, blasting, GOP, praising, :, ., Ar...       1
3  [portray, are, s, so, will, class, who, Romney...       2
4  [believe, do, n't, I, there, ., is, in, good, ...       2 
                                               Tweets  Labels
0  ["" "" Mockingbird "," Obama ",": "" Hosts ","...       0
1  [RT, @DS_CT, :, It, 's, a, sorry, state, of, a...       0
2  ['RT', '@ohgirlphrase', ':', 'US', 'has', '",'...       0
3  [RT, @ohgirlphrase, :, American, kid, ", You, ...       0
4  [RT, @ohgirlphrase, :, American, kid, ", You, ...       0 
 (1159, 2) 
 (1849, 2)


# Create "train_augmented" generator from "df_train_new" augmented data frame

In [ ]:
example_augmented = [data.Example.fromlist([df_train_new.Tweets[i],df_train_new.Labels[i]], fields) for i in range(df_train_new.shape[0])] 

train_augmented = data.Dataset(example_augmented, fields)

In [ ]:
train_augmented

In [ ]:
vars(train_augmented.examples[1])['labels']

0

An example from the augmented dataset:

In [ ]:
vars(train_augmented.examples[1])

{'labels': 0,
 'tweets': ['RT',
  '@DS_CT',
  ':',
  'It',
  "'s",
  'a',
  'sorry',
  'state',
  'of',
  'affairs',
  'when',
  'the',
  'US',
  'President',
  'menaces',
  'other',
  'branches',
  'of',
  'government',
  '.',
  'Obama',
  'mocks',
  'and',
  'menaces',
  'like',
  'governing',
  'is',
  'sport',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [ ]:
# Tweet.build_vocab(train)
# Label.build_vocab(train)

In [ ]:
# print('Size of input vocab : ', len(Tweet.vocab))
# print('Size of label vocab : ', len(Label.vocab))
# print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
# print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [ ]:
Tweet.build_vocab(train_augmented)
Label.build_vocab(train_augmented)

**Lots of stopwords!!**

In [ ]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  5431
Size of label vocab :  3
Top 10 words appreared repeatedly : [("'", 6537), (',', 5388), ('"', 3096), ('Obama', 1656), ('.', 1221), (':', 1206), ('#', 1183), ("''", 1130), ('the', 801), ('RT', 786)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Replace iterator -"train" with "train_augmented"
train_iterator, valid_iterator = data.BucketIterator.splits((train_augmented, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(4651, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,637,203 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [ ]:
# N_EPOCHS = 10
# best_valid_loss = float('inf')

# for epoch in range(N_EPOCHS):
     
#     # train the model
#     train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
#     # evaluate the model
#     valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
#     # save the best model
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'saved_weights.pt')
    
#     print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')



# Without Data Augmentation


  Train Loss: 1.065 | Train Acc: 49.98%
	 Val. Loss: 1.014 |  Val. Acc: 58.48% 

	Train Loss: 0.993 | Train Acc: 64.06%
	 Val. Loss: 0.949 |  Val. Acc: 64.73% 

	Train Loss: 0.927 | Train Acc: 69.12%
	 Val. Loss: 0.901 |  Val. Acc: 67.86% 

	Train Loss: 0.869 | Train Acc: 69.88%
	 Val. Loss: 0.857 |  Val. Acc: 73.21% 

	Train Loss: 0.824 | Train Acc: 74.87%
	 Val. Loss: 0.833 |  Val. Acc: 74.55% 

	Train Loss: 0.796 | Train Acc: 77.74%
	 Val. Loss: 0.820 |  Val. Acc: 75.89% 

	Train Loss: 0.776 | Train Acc: 78.67%
	 Val. Loss: 0.807 |  Val. Acc: 75.89% 

	Train Loss: 0.760 | Train Acc: 80.24%
	 Val. Loss: 0.802 |  Val. Acc: 76.79% 

	Train Loss: 0.746 | Train Acc: 81.67%
	 Val. Loss: 0.797 |  Val. Acc: 77.23% 

	Train Loss: 0.731 | Train Acc: 83.28%
	 Val. Loss: 0.791 |  Val. Acc: 77.68% 

# With Data Augmentation (20% of train samples)

In [ ]:
print('Results after data augmentation','\n')
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Results after data augmentation 

	Train Loss: 1.071 | Train Acc: 47.06%
	 Val. Loss: 1.000 |  Val. Acc: 66.96% 

	Train Loss: 0.968 | Train Acc: 67.47%
	 Val. Loss: 0.891 |  Val. Acc: 70.54% 

	Train Loss: 0.867 | Train Acc: 70.44%
	 Val. Loss: 0.833 |  Val. Acc: 71.88% 

	Train Loss: 0.823 | Train Acc: 73.94%
	 Val. Loss: 0.814 |  Val. Acc: 73.21% 

	Train Loss: 0.797 | Train Acc: 76.09%
	 Val. Loss: 0.796 |  Val. Acc: 76.34% 

	Train Loss: 0.781 | Train Acc: 77.98%
	 Val. Loss: 0.784 |  Val. Acc: 77.68% 

	Train Loss: 0.765 | Train Acc: 79.49%
	 Val. Loss: 0.776 |  Val. Acc: 77.68% 

	Train Loss: 0.750 | Train Acc: 81.32%
	 Val. Loss: 0.772 |  Val. Acc: 77.68% 

	Train Loss: 0.733 | Train Acc: 83.69%
	 Val. Loss: 0.762 |  Val. Acc: 79.02% 

	Train Loss: 0.719 | Train Acc: 85.20%
	 Val. Loss: 0.759 |  Val. Acc: 79.02% 



## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'